This will aggregate spotify data with features into daily summaries that we can use for correlations

In [79]:
import json
import pandas as pd

init_notebook_mode(connected=True)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_columns', None)


spotify_data = pd.read_csv("../data/spotify_history_w_features_nov21.csv")


spotify_data.head()

,Unnamed: 0,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,albumName,albumID
0,0,African Sun___Naïka,2021-01-01 10:26,Naïka,African Sun,4544,2021-01-01 10:26:00,0.802,0.4420,1.0,-8.908,1.0,0.4230,0.590,0.000023,0.107,0.4860,103.047,audio_features,7DQPyLJx2EVKlf5RTpK3Zo,spotify:track:7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/tracks/7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/audio-analysis/7DQPyLJx2EVKlf5RTpK3Zo,158486.0,4.0,"Lost in Paradïse, Pt. 1",1HgmA08XqxQ4hPrClSsLfi
1,1,I Feel Wealth and Abundance Flowing Toward Me Now___Instant Affirmations,2021-01-02 05:56,Instant Affirmations,I Feel Wealth and Abundance Flowing Toward Me Now,292197,2021-01-02 05:56:00,0.362,0.1400,0.0,-19.006,1.0,0.0467,0.797,0.014700,0.080,0.0396,87.819,audio_features,5SMNzzuRx7FDdvbo9NA3k6,spotify:track:5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/tracks/5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/audio-analysis/5SMNzzuRx7FDdvbo9NA3k6,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej
2,2,I Easily Bring Money and Wealth Into My Life___Instant Affirmations,2021-01-02 06:01,Instant Affirmations,I Easily Bring Money and Wealth Into My Life,312217,2021-01-02 06:01:00,0.290,0.0531,0.0,-20.613,1.0,0.0316,0.717,0.060500,0.104,0.0565,98.113,audio_features,3IcjAr7J6qe6wt52r6suqa,spotify:track:3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/tracks/3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/audio-analysis/3IcjAr7J6qe6wt52r6suqa,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej
3,3,I Am Aligned with the Energy of Wealth and Abundance___Instant Affirmations,2021-01-02 06:07,Instant Affirmations,I Am Aligned with the Energy of Wealth and Abundance,312217,2021-01-02 06:07:00,0.235,0.0829,0.0,-21.733,1.0,0.0327,0.673,0.490000,0.104,0.0371,103.446,audio_features,5FqidFOb9epSJZhKG9osPe,spotify:track:5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/tracks/5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/audio-analysis/5FqidFOb9epSJZhKG9osPe,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej
4,4,I Am Successful Strong and Satisfied___Instant Affirmations,2021-01-02 06:11,Instant Affirmations,I Am Successful Strong and Satisfied,244553,2021-01-02 06:11:00,0.433,0.1980,7.0,-15.202,1.0,0.0331,0.823,0.207000,0.164,0.2640,139.167,audio_features,6QHDfLwLSP9NnSMFAXdE8L,spotify:track:6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/tracks/6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/audio-analysis/6QHDfLwLSP9NnSMFAXdE8L,244553.0,4.0,Affirmations for Success,00b3RaOGzalm8f58l43QaU


In [80]:
from datetime import datetime
single_date = datetime.strptime(spotify_data.iloc[0]["endTime"], '%Y-%m-%d %H:%M')

# 
# we need to get start time .. endtime - milliseconds

In [81]:
# need to convert my timezone to an ISO format that get's location
def get_startTime(endTime: str, msPlayed: int):
    date_object = datetime.strptime(endTime, '%Y-%m-%d %H:%M')
    
    # multiply & divide by 1000 because msPlayed is milliseconds and .timestamp() is in seconds
    startTimestamp = ((date_object.timestamp() *  1000) - msPlayed) / 1000 
    
    startTime = datetime.fromtimestamp(startTimestamp)
    
    return startTime.strftime('%Y-%m-%d %H:%M')

spotify_data["startTime"] = spotify_data[["endTime", "msPlayed"]].apply(lambda x: get_startTime(x['endTime'], x['msPlayed']), axis=1)

type(spotify_data.iloc[0]['startTime'])

spotify_data

,Unnamed: 0,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,albumName,albumID,startTime
0,0,African Sun___Naïka,2021-01-01 10:26,Naïka,African Sun,4544,2021-01-01 10:26:00,0.802,0.4420,1.0,-8.908,1.0,0.4230,0.590,0.000023,0.1070,0.4860,103.047,audio_features,7DQPyLJx2EVKlf5RTpK3Zo,spotify:track:7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/tracks/7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/audio-analysis/7DQPyLJx2EVKlf5RTpK3Zo,158486.0,4.0,"Lost in Paradïse, Pt. 1",1HgmA08XqxQ4hPrClSsLfi,2021-01-01 10:25
1,1,I Feel Wealth and Abundance Flowing Toward Me Now___Instant Affirmations,2021-01-02 05:56,Instant Affirmations,I Feel Wealth and Abundance Flowing Toward Me Now,292197,2021-01-02 05:56:00,0.362,0.1400,0.0,-19.006,1.0,0.0467,0.797,0.014700,0.0800,0.0396,87.819,audio_features,5SMNzzuRx7FDdvbo9NA3k6,spotify:track:5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/tracks/5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/audio-analysis/5SMNzzuRx7FDdvbo9NA3k6,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:51
2,2,I Easily Bring Money and Wealth Into My Life___Instant Affirmations,2021-01-02 06:01,Instant Affirmations,I Easily Bring Money and Wealth Into My Life,312217,2021-01-02 06:01:00,0.290,0.0531,0.0,-20.613,1.0,0.0316,0.717,0.060500,0.1040,0.0565,98.113,audio_features,3IcjAr7J6qe6wt52r6suqa,spotify:track:3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/tracks/3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/audio-analysis/3IcjAr7J6qe6wt52r6suqa,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:55
3,3,I Am Aligned with the Energy of Wealth and Abundance___Instant Affirmations,2021-01-02 06:07,Instant Affirmations,I Am Aligned with the Energy of Wealth and Abundance,312217,2021-01-02 06:07:00,0.235,0.0829,0.0,-21.733,1.0,0.0327,0.673,0.490000,0.1040,0.0371,103.446,audio_features,5FqidFOb9epSJZhKG9osPe,spotify:track:5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/tracks/5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/audio-analysis/5FqidFOb9epSJZhKG9osPe,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 06:01
4,4,I Am Successful Strong and Satisfied___Instant Affirmations,2021-01-02 06:11,Instant Affirmations,I Am Successful Strong and Satisfied,244553,2021-01-02 06:11:00,0.433,0.1980,7.0,-15.202,1.0,0.0331,0.823,0.207000,0.1640,0.2640,139.167,audio_features,6QHDfLwLSP9NnSMFAXdE8L,spotify:track:6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/tracks/6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/audio-analysis/6QHDfLwLSP9NnSMFAXdE8L,244553.0,4.0,Affirmations for Success,00b3RaOGzalm8f58l43QaU,2021-01-02 06:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9753,9753,i tried___slowthai,2021-11-15 19:35,slowthai,i tried,89515,2021-11-15 19:35:00,0.428,0.7720,5.0,-8.158,0.0,0.4060,0.082,0.000000,0.3660,0.6100,137.250,audio_features,5SpZDtg5U1W35HNjq1xYrM,spotify:track:5SpZDtg5U1W35HNjq1xYrM,https://api.spotify.com/v1/tracks/5SpZDtg5U1W35HNjq1xYrM,https://api.spotify.com/v1/audio-analysis/5SpZDtg5U1W35HNjq1xYrM,134747.0,4.0,TYRON,7qE3WaOVAAIxgH8WtjbBBj,2021-11-15 19:33
9754,9754,Water No Get Enemy - Edit___Fela Kuti,2021-11-15 19:44,Fela Kuti,Water No Get Enemy - Edit,981,2021-11-15 19:44:00,0.641,0.9240,10.0,-7.697,0.0,0.0578,0.107,0.026900,0.0982,0.8910,90.858,audio_features,43ile6cBzr9uaC4bJf6J3N,spotify:track:43ile6cBzr9uaC4bJf6J3N,https://api.spotify.com/v1/tracks/43ile6cBzr9uaC4bJf6J3N,https://api.spotify.com/v1/audio-analysis/43ile6cBzr9uaC4bJf6J3N,591707.0,4.0,Best of The Black President,7325GfKum2hDK231i3LqA7,2021-11-15 19:43
9755,9755,No Buredi (No Bread)___Fela Kuti,2021-11-15 19:44,Fela Kuti,No Buredi (No Bread),0,2021-11-15 19:4

In [82]:
# get date of event in y-m-d format    
spotify_data.dropna()

def get_summaryday(startTime: str):
    date_object = datetime.strptime(startTime, '%Y-%m-%d %H:%M')
    
    return date_object.strftime('%Y-%m-%d')

spotify_data["summary_day"] = spotify_data["startTime"].apply(lambda x: get_summaryday(x))


spotify_data

,Unnamed: 0,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,albumName,albumID,startTime,summary_day
0,0,African Sun___Naïka,2021-01-01 10:26,Naïka,African Sun,4544,2021-01-01 10:26:00,0.802,0.4420,1.0,-8.908,1.0,0.4230,0.590,0.000023,0.1070,0.4860,103.047,audio_features,7DQPyLJx2EVKlf5RTpK3Zo,spotify:track:7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/tracks/7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/audio-analysis/7DQPyLJx2EVKlf5RTpK3Zo,158486.0,4.0,"Lost in Paradïse, Pt. 1",1HgmA08XqxQ4hPrClSsLfi,2021-01-01 10:25,2021-01-01
1,1,I Feel Wealth and Abundance Flowing Toward Me Now___Instant Affirmations,2021-01-02 05:56,Instant Affirmations,I Feel Wealth and Abundance Flowing Toward Me Now,292197,2021-01-02 05:56:00,0.362,0.1400,0.0,-19.006,1.0,0.0467,0.797,0.014700,0.0800,0.0396,87.819,audio_features,5SMNzzuRx7FDdvbo9NA3k6,spotify:track:5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/tracks/5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/audio-analysis/5SMNzzuRx7FDdvbo9NA3k6,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:51,2021-01-02
2,2,I Easily Bring Money and Wealth Into My Life___Instant Affirmations,2021-01-02 06:01,Instant Affirmations,I Easily Bring Money and Wealth Into My Life,312217,2021-01-02 06:01:00,0.290,0.0531,0.0,-20.613,1.0,0.0316,0.717,0.060500,0.1040,0.0565,98.113,audio_features,3IcjAr7J6qe6wt52r6suqa,spotify:track:3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/tracks/3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/audio-analysis/3IcjAr7J6qe6wt52r6suqa,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:55,2021-01-02
3,3,I Am Aligned with the Energy of Wealth and Abundance___Instant Affirmations,2021-01-02 06:07,Instant Affirmations,I Am Aligned with the Energy of Wealth and Abundance,312217,2021-01-02 06:07:00,0.235,0.0829,0.0,-21.733,1.0,0.0327,0.673,0.490000,0.1040,0.0371,103.446,audio_features,5FqidFOb9epSJZhKG9osPe,spotify:track:5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/tracks/5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/audio-analysis/5FqidFOb9epSJZhKG9osPe,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 06:01,2021-01-02
4,4,I Am Successful Strong and Satisfied___Instant Affirmations,2021-01-02 06:11,Instant Affirmations,I Am Successful Strong and Satisfied,244553,2021-01-02 06:11:00,0.433,0.1980,7.0,-15.202,1.0,0.0331,0.823,0.207000,0.1640,0.2640,139.167,audio_features,6QHDfLwLSP9NnSMFAXdE8L,spotify:track:6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/tracks/6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/audio-analysis/6QHDfLwLSP9NnSMFAXdE8L,244553.0,4.0,Affirmations for Success,00b3RaOGzalm8f58l43QaU,2021-01-02 06:06,2021-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9753,9753,i tried___slowthai,2021-11-15 19:35,slowthai,i tried,89515,2021-11-15 19:35:00,0.428,0.7720,5.0,-8.158,0.0,0.4060,0.082,0.000000,0.3660,0.6100,137.250,audio_features,5SpZDtg5U1W35HNjq1xYrM,spotify:track:5SpZDtg5U1W35HNjq1xYrM,https://api.spotify.com/v1/tracks/5SpZDtg5U1W35HNjq1xYrM,https://api.spotify.com/v1/audio-analysis/5SpZDtg5U1W35HNjq1xYrM,134747.0,4.0,TYRON,7qE3WaOVAAIxgH8WtjbBBj,2021-11-15 19:33,2021-11-15
9754,9754,Water No Get Enemy - Edit___Fela Kuti,2021-11-15 19:44,Fela Kuti,Water No Get Enemy - Edit,981,2021-11-15 19:44:00,0.641,0.9240,10.0,-7.697,0.0,0.0578,0.107,0.026900,0.0982,0.8910,90.858,audio_features,43ile6cBzr9uaC4bJf6J3N,spotify:track:43ile6cBzr9uaC4bJf6J3N,https://api.spotify.com/v1/tracks/43ile6cBzr9uaC4bJf6J3N,https://api.spotify.com/v1/audio-analysis/43ile6cBzr9uaC4bJf6J3N,591707.0,4.0,Best of The Black President,7325GfKum2hDK231i3LqA7,2021-11-15 19:43,2021-11-15
9755,9755,No Bu

In [101]:
##### generate a weighted percentage of summary values

# get total duration for songs listened on a given day
spotify_daily_play_summary = spotify_data.groupby(["summary_day"], as_index=False).agg({'msPlayed': 'sum'})
spotify_daily_play_summary.rename(columns={"msPlayed":"totalPlayDurationForDay"}, inplace=True)

# join with single entries so it can be a column value
spotify_data_combined = spotify_data.merge(spotify_daily_play_summary, how='left', on='summary_day')
spotify_data_combined["danceability_weight"] = 0

def get_weight(feature, msPlayed, totalPlayDurationForDay):
    return feature * (msPlayed/totalPlayDurationForDay)

spotify_data_combined["danceability"] = spotify_data_combined[["danceability", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["danceability"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["energy"] = spotify_data_combined[["energy", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["energy"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["key"] = spotify_data_combined[["key", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["key"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["loudness"] = spotify_data_combined[["loudness", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["loudness"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["mode"] = spotify_data_combined[["mode", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["mode"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["speechiness"] = spotify_data_combined[["speechiness", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["speechiness"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["acousticness"] = spotify_data_combined[["acousticness", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["acousticness"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["instrumentalness"] = spotify_data_combined[["instrumentalness", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["instrumentalness"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["liveness"] = spotify_data_combined[["liveness", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["liveness"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["valence"] = spotify_data_combined[["valence", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["valence"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined["tempo"] = spotify_data_combined[["tempo", "msPlayed", "totalPlayDurationForDay"]].apply(
                                    lambda x: get_weight(x["tempo"], x["msPlayed"], x["totalPlayDurationForDay"]), axis=1
                                )
spotify_data_combined

,Unnamed: 0,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,albumName,albumID,startTime,summary_day,totalPlayDurationForDay,danceability_weight
0,0,African Sun___Naïka,2021-01-01 10:26,Naïka,African Sun,4544,2021-01-01 10:26:00,0.802000,0.442000,1.000000,-8.908000,1.000000,0.423000,0.590000,2.350000e-05,0.107000,0.486000,103.047000,audio_features,7DQPyLJx2EVKlf5RTpK3Zo,spotify:track:7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/tracks/7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/audio-analysis/7DQPyLJx2EVKlf5RTpK3Zo,158486.0,4.0,"Lost in Paradïse, Pt. 1",1HgmA08XqxQ4hPrClSsLfi,2021-01-01 10:25,2021-01-01,4544,0
1,1,I Feel Wealth and Abundance Flowing Toward Me Now___Instant Affirmations,2021-01-02 05:56,Instant Affirmations,I Feel Wealth and Abundance Flowing Toward Me Now,292197,2021-01-02 05:56:00,0.021651,0.008373,0.000000,-1.136733,0.059809,0.002793,0.047668,8.791950e-04,0.004785,0.002368,5.252383,audio_features,5SMNzzuRx7FDdvbo9NA3k6,spotify:track:5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/tracks/5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/audio-analysis/5SMNzzuRx7FDdvbo9NA3k6,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:51,2021-01-02,4885487,0
2,2,I Easily Bring Money and Wealth Into My Life___Instant Affirmations,2021-01-02 06:01,Instant Affirmations,I Easily Bring Money and Wealth Into My Life,312217,2021-01-02 06:01:00,0.018533,0.003393,0.000000,-1.317316,0.063907,0.002019,0.045821,3.866376e-03,0.006646,0.003611,6.270111,audio_features,3IcjAr7J6qe6wt52r6suqa,spotify:track:3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/tracks/3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/audio-analysis/3IcjAr7J6qe6wt52r6suqa,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:55,2021-01-02,4885487,0
3,3,I Am Aligned with the Energy of Wealth and Abundance___Instant Affirmations,2021-01-02 06:07,Instant Affirmations,I Am Aligned with the Energy of Wealth and Abundance,312217,2021-01-02 06:07:00,0.015018,0.005298,0.000000,-1.388892,0.063907,0.002090,0.043009,3.131445e-02,0.006646,0.002371,6.610927,audio_features,5FqidFOb9epSJZhKG9osPe,spotify:track:5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/tracks/5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/audio-analysis/5FqidFOb9epSJZhKG9osPe,312217.0,4.0,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 06:01,2021-01-02,4885487,0
4,4,I Am Successful Strong and Satisfied___Instant Affirmations,2021-01-02 06:11,Instant Affirmations,I Am Successful Strong and Satisfied,244553,2021-01-02 06:11:00,0.021675,0.009911,0.350399,-0.760967,0.050057,0.001657,0.041197,1.036181e-02,0.008209,0.013215,6.966288,audio_features,6QHDfLwLSP9NnSMFAXdE8L,spotify:track:6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/tracks/6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/audio-analysis/6QHDfLwLSP9NnSMFAXdE8L,244553.0,4.0,Affirmations for Success,00b3RaOGzalm8f58l43QaU,2021-01-02 06:06,2021-01-02,4885487,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9753,9753,i tried___slowthai,2021-11-15 19:35,slowthai,i tried,89515,2021-11-15 19:35:00,0.009904,0.017864,0.115700,-0.188776,0.000000,0.009395,0.001897,0.000000e+00,0.008469,0.014115,3.175970,audio_features,5SpZDtg5U1W35HNjq1xYrM,spotify:track:5SpZDtg5U1W35HNjq1xYrM,https://api.spotify.com/v1/tracks/5SpZDtg5U1W35HNjq1xYrM,https://api.spotify.com/v1/audio-analysis/5SpZDtg5U1W35HNjq1xYrM,134747.0,4.0,TYRON,7qE3WaOVAAIxgH8WtjbBBj,2021-11-15 19:33,2021-11-15,3868404,0
9754,9754,Water No Get Enemy - Edit___Fela Kuti,2021-11-15 19:44,Fela Kuti,Water No Get Enemy - Edit,981,2021-11-15 19:44:00,0.000163,0.000234,0.002536,-0.001952,0.000000,0.000015,0.000027,6.821650e-0

In [102]:
# now let's get the daily summary for music listened to, summary here is the average of these values
spotify_data_summary = spotify_data_combined.groupby(["summary_day"], as_index=False).agg({
    'danceability': 'sum',
    "energy": 'sum',
    "key": "sum",
    "loudness": "sum",
    "mode": "sum",
    "speechiness": "sum",
    "acousticness": "sum",
    "instrumentalness": "sum",
    "liveness": "sum",
    "valence": "sum",
    "tempo": "sum"
})

spotify_data_summary

,summary_day,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2021-01-01,0.802000,0.442000,1.000000,-8.908000,1.000000,0.423000,0.590000,0.000023,0.107000,0.486000,103.047000
1,2021-01-02,0.384296,0.150550,4.075458,-19.512724,0.657844,0.085052,0.783974,0.255382,0.122638,0.158397,111.067440
2,2021-01-03,0.698208,0.608099,5.954831,-6.931956,0.590875,0.047173,0.436540,0.034609,0.133820,0.529980,113.489271
3,2021-01-04,0.626512,0.558016,4.510346,-7.996014,0.443771,0.109887,0.339395,0.106984,0.147277,0.277550,112.482724
4,2021-01-05,0.739367,0.698419,6.473555,-6.295132,0.590834,0.152500,0.345633,0.001803,0.140529,0.564154,115.935425
...,...,...,...,...,...,...,...,...,...,...,...,...
294,2021-11-11,0.501465,0.209352,5.681524,-20.926538,0.449606,0.106051,0.766519,0.595687,0.133447,0.396023,108.977240
295,2021-11-12,0.598041,0.422598,5.640821,-11.553548,0.605100,0.159091,0.576252,0.142573,0.138078,0.419132,114.608510
296,2021-11-13,0.637112,0.493408,5.400657,-9.085180,0.751847,0.156188,0.514747,0.012578,0.177804,0.483437,115.329783
297,2021-11-14,0.678973,0.601673,6.028165,-9.368776,0.459063,0.159315,0.341997,0.208572,0.129528,0.523914,111.875596


In [103]:
spotify_data_summary.to_csv("../data/spotify_data_summary_nov21_weighted.csv", index=False)